# Basket option implementation with Bachelier model CV

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pyfeng as pf

#import option_models as opt
from option_models import basket

In [3]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [4]:
cor_m

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [5]:
print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket)

[1 0 0 0]
26.570984651213593


In [6]:
# Compare the price to normal model formula

norm1 = pf.Norm(sigma=40)
price_norm = norm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_norm)

26.570984651213593 26.570845957870507


In [7]:
# A trivial test case 2
# all assets almost perfectly correlated:
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 5
strike = 120

print( cor_m )

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_norm)

[[1.     0.9999 0.9999 0.9999]
 [0.9999 1.     0.9999 0.9999]
 [0.9999 0.9999 1.     0.9999]
 [0.9999 0.9999 0.9999 1.    ]]
26.57211110181949 26.570845957870507


In [8]:
# A full test set for basket option with exact price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 100
price_exact = 28.0073695

In [9]:
weights, cor_m

(array([0.25, 0.25, 0.25, 0.25]),
 array([[1. , 0.5, 0.5, 0.5],
        [0.5, 1. , 0.5, 0.5],
        [0.5, 0.5, 1. , 0.5],
        [0.5, 0.5, 0.5, 1. ]]))

In [20]:
price_basket = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, bsm=False)
print(price_basket, price_exact)

19.36087718554104 8.5132252


# [To Do] Basket option implementation based on BSM model
## Write the similar test for BSM

In [21]:
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)
print(price_basket)

3.866780912452648


In [22]:
# A trivial test case 1: 
# one asset have 100% weight (the others zero)
# the case should be equivalent to the BSM or Normal model price

spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 0, 0, 0])
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

print(weights)

np.random.seed(123456)
price_basket = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, bsm=True)

[1 0 0 0]


In [23]:
bsm1 = pf.Bsm(sigma=vol[0])
price_bsm = bsm1.price(strike, spot[0], texp, cp=1)
print(price_basket, price_bsm)

7.068690377897586 28.713486748445934


# Spread option implementation based on normal model

In [24]:
# A full test set for spread option

spot = np.array([100, 96])
vol = np.array([0.2, 0.1])
weights = np.array([1, -1])
divr = np.array([1, 1])*0.05
intr = 0.1
cor = 0.5
cor_m = np.array([[1, cor], [cor, 1]])
texp = 1
strike = 0
price_exact = 8.5132252

In [25]:
# MC price based on normal model
# make sure that the prices are similar

np.random.seed(123456)
price_spread = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=False)
print(price_spread, price_exact)

8.317680907159142 8.5132252


# Spread option implementation based on BSM model

In [26]:
# Once the implementation is finished the BSM model price should also work
price_spread = basket.basket_price_mc(
    strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, bsm=True)
price_spread

2.3844932771058997e+31

In [17]:
# You also test Kirk's approximation
kirk = pf.BsmSpreadKirk(vol, cor=cor, divr=divr, intr=intr)
price_kirk = kirk.price(strike, spot, texp)
print(price_kirk, price_spread)

8.513225229545505 2.3844932771058997e+31


# [To Do] Complete the implementation of basket_price_norm_analytic
# Compare the MC stdev of BSM basket prices from with and without CV

In [76]:
# The basket option example from above
spot = np.ones(4) * 100
vol = np.ones(4) * 0.4
weights = np.array([1, 1, 1, 1])/4
divr = np.zeros(4)
intr = 0
cor_m = 0.5*np.identity(4) + 0.5
texp = 5
strike = 120

In [84]:
### Make sure that the analytic normal price is correctly implemented
basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr)

52.8320029105732

In [85]:
# Run below about 100 times and get the mean and stdev

### Returns 2 prices, without CV and with CV 
price_basket = basket.basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m)

In [87]:
print(price_basket)

[241.00922392 222.20173223]
